# **Import Libraries**

In [ ]:
! conda install -c conda-forge gdcm -y;

In [ ]:
import sys
sys.path.append("../input/timmeffnetv2")

import platform
import numpy as np
import pandas as pd
import os
from tqdm.notebook import tqdm
import cv2
import pydicom
import glob
import gc
from math import ceil
import matplotlib.pyplot as plt
from pydicom.pixel_data_handlers.util import apply_voi_lut
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda.amp import GradScaler, autocast
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
import seaborn as sn

import warnings
warnings.simplefilter('ignore')

# **Load Metadata**

In [ ]:
train_image = pd.read_csv("../input/siim-covid19-detection/train_image_level.csv")
train_study = pd.read_csv("../input/siim-covid19-detection/train_study_level.csv")

In [ ]:
TRAIN_DIR = "../input/siim-covid19-detection/train/"
train_study['StudyInstanceUID'] = train_study['id'].apply(lambda x: x.replace('_study', ''))
train = train_image.merge(train_study, on='StudyInstanceUID')

# Make a path folder
paths = []
for instance_id in tqdm(train['StudyInstanceUID']):
    paths.append(glob.glob(os.path.join(TRAIN_DIR, instance_id +"/*/*"))[0])

train['path'] = paths

train = train.drop(['id_x', 'id_y'], axis=1)

train.head()

# **Exploratory Analysis**

In [ ]:
one_hot_encode_target = train.iloc[:, 3:7]
labels = one_hot_encode_target.columns
counts_classes = one_hot_encode_target.sum(axis = 0)
perc_counts = 100 * counts_classes / counts_classes.sum()

In [ ]:
plt.figure(figsize = (16, 10))

#define Seaborn color palette to use
colors = sn.color_palette('pastel')[0:5]

#create pie chart
_ = plt.pie(perc_counts, labels = labels, colors = colors, autopct='%.0f%%')

# **Load DIACOM Dataset**

In [ ]:
def dicom2array(path, voi_lut=True, fix_monochrome=True):
    dicom = pydicom.read_file(path)
    # VOI LUT (if available by DICOM device) is used to
    # transform raw DICOM data to "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return data

In [ ]:
class Config:
    image_size = (224, 224)
    train_bs = 32
    valid_bs = 16
    num_workers = 8
    num_finetuning_epochs = 10
    num_total_epochs = 25
    scaler = GradScaler()

In [ ]:
fig, ax = plt.subplots(1,5,figsize = (25,10))

for i, path in enumerate(train['path'][0:5]):
    image = dicom2array(path)
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    image = cv2.resize(image, (400, 400))
    ax[i].imshow(image)

In [ ]:
class SIIMData(Dataset):
    def __init__(self, df, is_train = True, augments = None, img_size = Config.image_size):
        super().__init__()
        self.df = df
        self.is_train = is_train
        self.augments = augments
        self.img_size = img_size
        
    def __getitem__(self, idx):
        image_id = self.df['StudyInstanceUID'].values[idx] 
        image_path = self.df['path'].values[idx]
        image = dicom2array(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        image = cv2.resize(image, Config.image_size)
        
        # Augments must be albumentations
        if self.augments:
            image = self.augments(image=image)['image']
        else:
            image = torch.tensor(image, dtype=torch.float)
        
        if self.is_train:
            label = self.df[self.df['StudyInstanceUID'] == image_id].values.tolist()[0][3:7]
            return image, torch.tensor(label)
        
        return image
    
    def __len__(self):
        return len(self.df)

In [ ]:
train_transform = A.Compose({
        A.HorizontalFlip(p=0.5),
        A.Rotate(limit=(-40, 40)),
        A.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        })

valid_test_transform = A.Compose({
        A.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        })

# **Splitting the Dataset** 

In [ ]:
# train["class"] = train.iloc[:, 3:7].to_numpy().argmax(axis = 1)

# train = train[:150]
# np.unique(train["class"].to_numpy(), return_counts = True)

In [ ]:
train["class"] = train.iloc[:, 3:7].to_numpy().argmax(axis = 1)

train, valid = train_test_split(train, stratify = train["class"], shuffle = True, test_size=0.3)

valid, test = train_test_split(valid, stratify = valid["class"], shuffle = True, test_size=0.5)

In [ ]:
train_data = SIIMData(train, augments = train_transform)
valid_data = SIIMData(valid, augments = valid_test_transform)
test_data = SIIMData(test, augments = valid_test_transform)

In [ ]:
_, class_sample_count = np.unique(train["class"].to_numpy(), return_counts = True)
#Balance the Training Dataset

weight = 1. / class_sample_count
samples_weight = np.array([weight[t] for t in train["class"]])

samples_weight = torch.from_numpy(samples_weight)
samples_weight = samples_weight.double()
sampler = WeightedRandomSampler(samples_weight, len(samples_weight))

In [ ]:
train_loader = DataLoader(dataset = train_data, batch_size = Config.train_bs, num_workers = Config.num_workers, sampler = sampler)
valid_loader = DataLoader(dataset = valid_data, batch_size = Config.valid_bs, num_workers = Config.num_workers)
test_loader = DataLoader(dataset = test_data, batch_size = Config.valid_bs, num_workers = Config.num_workers)

# **DenseNet201**

In [ ]:
# There is a bug in pytorch when downloading .pt through wget :(
# ! wget -c https://github.com/PedroRASB/COVID-19-Twice-Transfer-DNNs/blob/master/NetC.pt

In [ ]:
class DenseNet201(nn.Module):

    def __init__(self, num_classes = 4):
        super(DenseNet201, self).__init__()
        self.model = torch.load("../input/covidnet/NetC.pt", map_location=torch.device('cpu'))
        
        #Freezing the loaded model
        for param in self.model.parameters():
            param.requires_grad = False
            
        #The output layer is unfreeze
        self.model.classifier = nn.Linear(self.model.classifier.in_features, num_classes)
        
    def unfreeze_all_layers(self):
        for param in self.model.parameters():
            param.requires_grad = True
        
    def forward(self, x):
        x = self.model(x)
        return x

# **Training the model**

In [ ]:
class Trainer:
    def __init__(self, model, train_loader, valid_loader, optimizer, criterion, device):
        """
        Constructor for Trainer class
        """
        self.model = model
        self.train = train_loader
        self.valid = valid_loader
        self.optim = optimizer
        self.criterion = criterion
        self.device = device
        self.scaler = GradScaler()
    
    def train_one_epoch(self):
        """
        Runs one epoch of training, backpropagation and optimization
        """
        self.model.train()
        
        running_loss = 0
        running_acc = 0
        
        for xtrain, ytrain in self.train:
            xtrain = xtrain.to(self.device).float()
            ytrain = ytrain.to(self.device).float()
            xtrain = xtrain.permute(0, 3, 1, 2)
            
            with autocast():
                # Get predictions
                z = self.model(xtrain)

                # Training
                train_loss = self.criterion(z, ytrain)
                self.scaler.scale(train_loss).backward()
                
                self.scaler.step(self.optim)
                self.scaler.update()
                self.optim.zero_grad(set_to_none=True)

                # For averaging and reporting later
                running_loss += train_loss

                # Convert the predictions and corresponding labels to right form
                train_preds = torch.argmax(z, 1).detach().cpu().numpy()
                train_labels = ytrain.detach().cpu().numpy().argmax(axis = 1)
                
                running_acc += ( train_preds == train_labels ).sum()                
        
        # Now average the running loss over all batches and return
        running_loss /= train.shape[0]
        running_acc  /= train.shape[0]
        print(f"Training Loss: {running_loss:.4f}")
        print(f"Training Accuracy: {running_acc:.4f}\n")
        
        # Free up memory
        del train_preds, train_labels, xtrain, ytrain, z
        gc.collect()
        torch.cuda.empty_cache()
        
        return (running_loss, running_acc)

    def valid_one_epoch(self):
        """
        Runs one epoch of prediction
        """        
        model.eval()
                
        with torch.no_grad():
            
            running_loss = 0
            running_acc = 0
            
            for xval, yval in self.valid:
                xval = xval.to(self.device).float()
                yval = yval.to(self.device).float()
                xval = xval.permute(0, 3, 1, 2)
                
                val_z = self.model(xval)
                
                val_loss = self.criterion(val_z, yval)
                
                running_loss += val_loss.item()
                
                val_preds = torch.argmax(val_z, 1).detach().cpu().numpy()
                val_labels = yval.detach().cpu().numpy().argmax(axis = 1)
                
                running_acc += ( val_preds == val_labels ).sum()
            
            # Get the final loss
            running_loss /= valid.shape[0]
            running_acc  /= valid.shape[0]
            
            print(f"Validation Loss: {running_loss:.4f}")
            print(f"Validation Accuracy: {running_acc:.4f}")
            
            # Free up memory
            del val_labels, val_preds, xval, yval, val_z
            gc.collect()
            torch.cuda.empty_cache()
            
        return (running_loss, running_acc)

In [ ]:
# Training Code
print(f"[INFO] Training on {train.shape[0]} samples and validation on {valid.shape[0]} samples")

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model = DenseNet201().to(device)
    
# optim = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.001)

optim = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.01)

criterion = nn.BCEWithLogitsLoss()

trainer = Trainer(
    model=model,
    train_loader=train_loader,
    valid_loader=valid_loader,
    optimizer=optim,
    criterion=criterion,
    device=device,
)

train_losses = []
valid_losses = []

train_acc = []
valid_acc = []

best_acc = 0

for epoch in range(Config.num_finetuning_epochs):
    print(f"{'-'*25} EPOCH: {epoch+1}/{Config.num_total_epochs} {'-'*25}")

    # Run one training epoch
    current_train_loss, current_train_acc = trainer.train_one_epoch()
    train_losses.append(current_train_loss)
    train_acc.append(current_train_acc)

    # Run one validation epoch
    current_val_loss, current_val_acc = trainer.valid_one_epoch()
    valid_losses.append(current_val_loss)
    valid_acc.append(current_val_acc)
    
    if(current_val_acc > best_acc):
        best_acc = current_val_acc
        torch.save(trainer.model, 'best-model.pt')

    # Empty CUDA cache
    torch.cuda.empty_cache()

# **Evaluate the model**

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

fig, ax = plt.subplots(1,2,figsize = (25,10))

ax[0].plot(train_losses, label='Training loss')
ax[0].plot(valid_losses, label='Validation loss')
ax[0].set_xlabel("Epochs")
ax[0].set_ylabel("Loss")
ax[0].legend(frameon=False)

ax[1].plot(train_acc, label='Training accuracy')
ax[1].plot(valid_acc, label='Validation accuracy')
ax[1].set_xlabel("Epochs")
ax[1].set_ylabel("Accuracy")
ax[1].legend(frameon=False)

# **Finetuning**

In [ ]:
# Training Code -- Finetuning
print("\nFinetuning\n")

trainer.model.unfreeze_all_layers()
trainer.optimizer = torch.optim.SGD(trainer.model.parameters(), lr=0.0001, momentum=0.9, weight_decay=0.01)

for epoch in range(Config.num_finetuning_epochs, Config.num_total_epochs):
    print(f"{'-'*25} EPOCH: {epoch+1}/{Config.num_total_epochs} {'-'*25}")

    # Run one training epoch
    current_train_loss, current_train_acc = trainer.train_one_epoch()
    train_losses.append(current_train_loss)
    train_acc.append(current_train_acc)

    # Run one validation epoch
    current_val_loss, current_val_acc = trainer.valid_one_epoch()
    valid_losses.append(current_val_loss)
    valid_acc.append(current_val_acc)
    
    if(current_val_acc > best_acc):
        best_acc = current_val_acc
        torch.save(trainer.model, 'best-model.pt')

    # Empty CUDA cache
    torch.cuda.empty_cache()

# **Evaluate the model**

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

fig, ax = plt.subplots(1,2,figsize = (25,10))

ax[0].plot(train_losses, label='Training loss')
ax[0].plot(valid_losses, label='Validation loss')
ax[0].set_xlabel("Epochs")
ax[0].set_ylabel("Loss")
ax[0].legend(frameon=False)

ax[1].plot(train_acc, label='Training accuracy')
ax[1].plot(valid_acc, label='Validation accuracy')
ax[1].set_xlabel("Epochs")
ax[1].set_ylabel("Accuracy")
ax[1].legend(frameon=False)

In [ ]:
# test-the-model
model.eval()  # it-disables-dropout

confusion_matrix = np.zeros((4,4))

with torch.no_grad():

    for images, labels in test_loader:
        
        images = images.to(device).float()
        labels = labels.to(device).float()
        images = images.permute(0, 3, 1, 2)
        
        outputs = model(images)
        test_preds = torch.argmax(outputs, 1).detach().cpu().numpy()
        test_labels = labels.detach().cpu().numpy().argmax(axis = 1)
            
        for i in range(test_preds.shape[0]):
            confusion_matrix[test_preds[i], test_labels[i]] += 1

# Save 
#torch.save(model.state_dict(), 'model.ckpt')

In [ ]:
confusion_matrix

In [ ]:
import seaborn as sn

labels = ["Negative\nfor\nPneumonia", "Typical\nAppearance", "Indeterminate\nAppearance", "Atypical\nAppearance"]

df_cm = pd.DataFrame(confusion_matrix, labels, labels)
plt.figure(figsize=(10,7))
sn.set(font_scale=1.4) # for label size
ax = sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}) # font size

ax.set_xlabel("True Labels")
ax.set_ylabel("Predicted Labels")
ax.xaxis.set_label_position('top')
plt.tick_params(axis='both', which='major', labelsize=10, labelbottom = False, bottom=False, top = False, labeltop=True)